# IMAGE PROCESSING #

1. Import Texas School geo data
2. Create 30cm TIF file for each school in Austin ISD
3. ZERO SHOT BUILDING DETECTION (completed in another Studio)
4. Get TIF for each bounding box of the buildings detected

In [11]:
import os
import numpy as np
import pandas as pd
from PIL import Image
import geopandas as gpd
from tqdm.auto import tqdm
import leafmap

In [12]:
rooftop_root_folder = os.getcwd()
data_folder = os.path.join(rooftop_root_folder, 'data')
images_folder = os.path.join(data_folder, 'images')
schools_folder = os.path.join(images_folder, 'schools')
school_geojson_file = os.path.join(data_folder, 'school_geocodes.geojson')
school_csv_file = os.path.join(data_folder, 'tx_schools.csv')

Filter Texas School Data - currenlty focused on Austin ISD

In [13]:
schools_geo = gpd.read_file('data/school_geocodes.geojson')
schools = pd.read_csv('data/tx_schools.csv', index_col=0)
schools = pd.concat([schools_geo, schools], axis=1)


In [14]:
gdf_aisd = schools[schools['School District'] == 'Austin ISD'].copy().reset_index(drop=True)
gdf_aisd = gdf_aisd[~(gdf_aisd.geometry == None)].copy()

Generate TIF files for each school

In [15]:
def generate_tif_files(gdf):
    bbox_scaler = np.array([-0.00326218, -0.0018414 ,  0.00233782,  0.0020586 ])
    for i, school in gdf_aisd.iterrows():
        tif_filename = f'{school["School Name"]}.tif'
        center = school.geometry.centroid.coords[0]

        bbox = (np.array(bbox_scaler) + np.repeat(np.array(center).reshape(1,2), 2, axis=0).flatten()).tolist()
        leafmap.map_tiles_to_geotiff(
            output=os.path.join(schools_folder, tif_filename), bbox=bbox, resolution=.03, source="Satellite", overwrite=True
        )

# generate_tif_files(gdf_aisd)
# m = leafmap.Map(center=center[::-1], zoom=18, height="800px")
# m.add_basemap('SATELLITE')

Generate TIF files for each building located in each school (this is achieved using geosam in another studio)

In [16]:
def tif_to_png(file_path, tif_file):
  tiff_image = Image.open(os.path.join(file_path, tif_file))

  # Convert the image to JPEG format
  jpeg_image = tiff_image.convert('RGB')

  # Save the JPEG image
  jpeg_image.save(os.path.join(file_path, tif_file.replace('.tif', '.png')))

In [17]:
tifs = [f for f in os.listdir(schools_folder) if f.endswith('.tif')]

In [ ]:
for tif in tqdm(tifs):
    tif_to_png(schools_folder, tif)

In [ ]:
geojsons = [f for f in os.listdir('../geosam/data') if f.endswith('.geojson')]

In [ ]:
for geojson in tqdm(geojsons):
    if not 'Akins' in geojson:
        gdf = gpd.GeoDataFrame.from_file(os.path.join('../geosam/data', geojson))
        school_name = geojson.split('_')[0]
        school_path = os.path.join(schools_folder, school_name)
        os.makedirs(school_path, exist_ok=True)
        os.makedirs(os.path.join(school_path, 'tif'), exist_ok=True)
        os.makedirs(os.path.join(school_path, 'png'), exist_ok=True)
        for i, bounds in tqdm(gdf.bounds.iterrows()):
            tif_filename = f'{school_name} Building {i}.tif'
            bbox = bounds.values.tolist()

            leafmap.map_tiles_to_geotiff(
                output=os.path.join(school_path, 'tif', tif_filename), bbox=bbox, resolution=.03, source="Satellite", overwrite=True
            )
            tif_to_png(school_path, tif_filename)

Convert each building TIF to png for use in Roboflow inference

In [3]:
from PIL import Image

